# 📞 AI-Powered Call Center Intelligence Demo w/ Snowflake Cortex

`NOTE!` If you're using the provided sample do / do not have audio. Skip to the `SET_UP` cell.

## 🎯 Purpose
This notebook demonstrates how contact center teams leverage Snowflake's Cortex AI capabilities to transform raw call transcripts into actionable customer intelligence. Experience conversational AI that protects customer privacy, extracts critical information, triages calls by urgency, and analyzes sentiment—all while maintaining compliance and security.

## 💡 Why do this?
- 🔒 **Privacy Protection**: Automatic PII removal to ensure compliance with data protection regulations
- 📊 **Intelligent Extraction**: AI-driven entity extraction to capture key customer information and call details
- 🎯 **Smart Triaging**: Automated call classification to route issues to the right teams
- 😊 **Sentiment Analysis**: Real-time customer satisfaction insights to identify at-risk accounts
- 📈 **Quality Assurance**: Identify coaching opportunities and best practices
- ⚡ **Response Acceleration**: Generate contextual responses to reduce handle time

## 🛠️ Solution Components
Note: This demo uses SQL & Python to showcase Snowflake Cortex capabilities for call center intelligence

1. 🔒 **Privacy-First Processing**
   - Automatic PII detection and redaction using `AI_COMPLETE`
   - Structured output for compliance tracking
   - Secure data handling throughout the pipeline

2. 🤖 **AI-Powered Call Analysis**
   - Entity extraction with `AI_EXTRACT` for customer details, issues, and resolutions
   - Multi-dimensional call classification with `AI_CLASSIFY`
   - Customer sentiment scoring with `AI_SENTIMENT`
   - Agent performance assessment

3. ⚙️ **Automated Intelligence Pipeline**
   - Real-time processing of new call transcripts
   - Automatic quality scoring and alert generation
   - Trend detection and pattern recognition

4. 🎯 **Interactive Call Center Dashboard**
   - Natural language queries for call analysis
   - Agent performance metrics and coaching insights
   - Customer satisfaction trends and risk indicators

## 🌟 Common Scenarios
*"Show me all calls where customers were dissatisfied with billing issues."*

*"Which agents have the highest customer satisfaction scores this month?"*

*"Identify calls that need urgent follow-up based on sentiment and issue type."*

*"Generate a response template for common roaming charge complaints."*


In [ ]:
# Core packages for call center analytics and visualization
import streamlit as st
import pandas as pd
import altair as alt
import json

# Snowflake session for AI-powered analytics
from snowflake.snowpark.context import get_active_session

session = get_active_session()


## Cortex powered Transcription:
The first step in our call center intelligence pipeline is converting audio recordings into searchable, analysable text using `AI_TRANSCRIBE`. This powerful function:

- Automatically transcribes audio files into text
- Identifies different speakers in the conversation
- Captures timestamps for precise analysis
- Maintains high accuracy across accents and audio quality

You can configure the settings to adjust for:
- **Speaker Labels**: Identifies customer vs agent segments
- **Timestamp Granularity**: Speaker-level timing for detailed analysis
- **Audio Support**: Handles common call recording formats
- **Scalable Processing**: Automatically processes new uploads

In [ ]:
CREATE SCHEMA IF NOT EXISTS AI_SOL.CALL_CENTERS;

CREATE STAGE IF NOT EXISTS AI_SOL.CALL_CENTERS.audio_files_stage -- if you already have an external file location (eg. S3) with audios, point this there.
    DIRECTORY = ( ENABLE = true )
    ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

LIST @AI_SOL.CALL_CENTERS.audio_files_stage; -- if you haven't already, place your audio files here.

In [ ]:
CREATE TABLE IF NOT EXISTS AI_SOL.CALL_CENTERS.transcribed_data AS (
SELECT
TO_FILE('@AI_SOL.CALL_CENTERS.audio_files_stage', RELATIVE_PATH) AS audio_file,
AI_TRANSCRIBE (TO_FILE('@AI_SOL.CALL_CENTERS.audio_files_stage', RELATIVE_PATH),
    {'timestamp_granularity': 'speaker'}) AS raw_transcript -- note: you have the option of word level timestamps, speaker, or none
FROM DIRECTORY(@AI_SOL.CALL_CENTERS.audio_files_stage));

In [ ]:
# Call center data configuration
transcripts_table = 'AI_SOL.CALL_CENTERS.TRANSCRIBED_DATA'

# Display table location for reference
st.markdown(f"""
**Call Center Data Sources:**
- 📞 Call Transcripts: `{transcripts_table}`
  - AUDIO_FILE: Metadata about the audio recording
  - RAW_TRANSCRIPT: Full conversation transcript with speaker labels
""")

# Quick data preview
st.markdown("### 📊 Data Preview")
preview_df = session.table(transcripts_table).limit(3).to_pandas()
st.dataframe(preview_df, use_container_width=True)


# 🔒 Privacy-First Processing: PII Removal with AI_COMPLETE

Before we can analyze call data, we must ensure customer privacy and regulatory compliance. Using `AI_COMPLETE` with structured outputs, we can intelligently identify and redact Personally Identifiable Information (PII) while maintaining the context needed for analysis.

This approach goes beyond simple pattern matching—it understands context to avoid false positives and ensures comprehensive PII protection.

## 🎯 What We'll Redact:
- **Customer Names**: Full names mentioned in conversations
- **Phone Numbers**: Mobile and landline numbers
- **Account Numbers**: Customer account identifiers
- **Addresses**: Physical locations and postcodes
- **Payment Information**: Credit card details, bank information
- **Personal Identifiers**: Dates of birth, email addresses


In [ ]:
-- AI-powered PII removal with structured tracking
-- This demonstrates intelligent PII detection and redaction while maintaining audit trails for compliance

-- NOTE: the converted audio is semi-structured, you're still able to access and operate on that data.

SELECT 
    AUDIO_FILE,
    RAW_TRANSCRIPT as original_transcript,
    AI_COMPLETE(
        model => 'claude-sonnet-4-5',
        prompt => 'You are a data privacy expert. Analyze the following call transcript and redact all PII while preserving the conversation context and business value. Replace PII with generic placeholders like [CUSTOMER_NAME], [PHONE_NUMBER], [ACCOUNT_NUMBER], [ADDRESS], [POSTCODE], [EMAIL], [DATE_OF_BIRTH], etc.' ||
                  '\n\nCALL TRANSCRIPT:\n' || 
                  RAW_TRANSCRIPT || 
                  '\n\nProvide the redacted transcript and list all PII types found.',
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'redacted_transcript': {
                        'type': 'string',
                        'description': 'The full transcript with all PII replaced by generic placeholders'
                    },
                    'pii_types_found': {
                        'type': 'array',
                        'items': {'type': 'string'},
                        'description': 'List of PII types detected (e.g., customer_name, phone_number, account_number)'
                    },
                    'pii_count': {
                        'type': 'integer',
                        'description': 'Total number of PII instances redacted'
                    },
                    'contains_sensitive_financial': {
                        'type': 'boolean',
                        'description': 'Whether the call contained sensitive financial information'
                    }
                },
                'required': ['redacted_transcript', 'pii_types_found', 'pii_count', 'contains_sensitive_financial'],
                'additionalProperties': false
            }
        }
    ) as pii_analysis
FROM {{transcripts_table}}
LIMIT 5;


# 📊 Entity Extraction: Capturing Key Call Information with AI_EXTRACT

Now that we've protected customer privacy, let's extract the valuable business intelligence from our calls. Using `AI_EXTRACT`, we can automatically identify and structure key information from unstructured conversation transcripts.

This transforms hours of manual call review into instant, structured insights.

## 🎯 What We'll Extract:
- **Agent Information**: Name, department, performance indicators
- **Customer Details**: Issue type, account status, contact reason
- **Call Outcomes**: Resolution status, follow-up required, actions taken
- **Product/Service**: Specific products or services discussed
- **Financial Details**: Charges, refunds, pricing discussed (amounts only, no account numbers)
- **Call Metadata**: Duration, escalation status, callback scheduled


In [ ]:
-- AI-powered entity extraction from call transcripts with Agent QA Focus
-- This demonstrates intelligent information extraction
-- to structure unstructured conversation data with emphasis on agent performance

SELECT 
    AUDIO_FILE:RELATIVE_PATH::string as call_id,
    RAW_TRANSCRIPT:audio_duration::float as call_duration,
    AI_EXTRACT(
        text => RAW_TRANSCRIPT:text::string,
        responseFormat => [
            ['agent_name', 'What is the name of the customer service agent handling this call?'],
            ['agent_greeting_quality', 'How would you rate the quality of the agent greeting and opening? (Professional, Adequate, Poor, or Missing)'],
            ['agent_professionalism', 'Did the agent maintain a professional and courteous tone throughout the entire call? (Excellent, Good, Fair, or Poor)'],
            ['agent_empathy', 'Did the agent demonstrate empathy and active listening? Provide specific examples or quotes from the transcript.'],
            ['agent_knowledge', 'Did the agent demonstrate adequate product and service knowledge? Were there any knowledge gaps or hesitation?'],
            ['agent_problem_solving', 'How effectively did the agent troubleshoot and resolve the issue? Rate as Excellent, Good, Fair, or Poor and explain.'],
            ['agent_communication_clarity', 'Was the agent communication clear, concise, and easy to understand? Provide examples of good or poor communication.'],
            ['agent_hold_handling', 'If the customer was placed on hold, did the agent explain why, ask permission, and provide updates? (Yes/No/Not Applicable)'],
            ['agent_closing_quality', 'Did the agent properly summarize the resolution, confirm customer understanding, and ensure satisfaction before closing? (Yes/No/Partial)'],
            ['agent_compliance', 'Did the agent follow required compliance steps like identity verification or providing disclosures? (Yes/No/Partial)'],
            ['customer_issue_type', 'What was the primary reason for the call? (e.g., billing, technical support, account inquiry, complaint, upgrade)'],
            ['issue_description', 'Provide a brief but comprehensive description of the customer issue or request.'],
            ['resolution_provided', 'Was the customer issue fully resolved during this call? (True/False)'],
            ['resolution_details', 'Describe how the issue was resolved or what next steps were provided to the customer.'],
            ['products_services_mentioned', 'List all products or services discussed during the call (e.g., data plans, devices, roaming, insurance).'],
            ['financial_amounts', 'List any monetary amounts discussed including charges, refunds, credits, or prices mentioned.'],
            ['follow_up_required', 'Is follow-up action required after this call? (True/False)'],
            ['follow_up_details', 'If follow-up is required, what specific actions need to be taken and by when?'],
            ['customer_satisfaction_indicators', 'What specific phrases, words, or tone from the customer indicated their level of satisfaction or dissatisfaction?'],
            ['escalation_occurred', 'Was this call escalated to a supervisor or did the customer request escalation? (True/False)'],
            ['escalation_reason', 'If escalated, what was the specific reason for the escalation?'],
            ['call_outcome', 'What was the overall outcome of the call? (resolved, pending_followup, escalated, callback_scheduled, unresolved)'],
            ['qa_score_recommendation', 'Based on the agent overall performance, what QA score (1-10) would you recommend and why?'],
            ['agent_strengths', 'What specific strengths did this agent demonstrate during the call?'],
            ['coaching_opportunities', 'What specific coaching opportunities or improvement areas exist for this agent? Be detailed and actionable.']
        ]
    ) as extracted_entities
FROM {{transcripts_table}}
LIMIT 5;

# 🎯 Smart Call Triaging with AI_CLASSIFY

Effective call center operations require intelligent routing and prioritization. Using `AI_CLASSIFY`, we can automatically categorize calls by urgency, department, and issue complexity to ensure the right resources handle each customer interaction.

This goes beyond keyword matching—it understands context, tone, and business impact to make intelligent routing decisions.

## 🎯 Classification Dimensions:
- **Urgency Level**: Critical, High, Medium, Low
- **Department**: Billing, Technical Support, Sales, Retention, Complaints
- **Complexity**: Simple, Moderate, Complex, Escalation Required
- **Customer Risk**: At-Risk, Neutral, Satisfied, Promoter


In [ ]:
-- Multi-dimensional call classification
-- This demonstrates intelligent call routing and prioritization

SELECT 
    AUDIO_FILE:RELATIVE_PATH::string as call_id,
    RAW_TRANSCRIPT:text::string as transcript_text,
    -- Classify by urgency level
    AI_CLASSIFY(
        RAW_TRANSCRIPT:text::string,
        [
            {'label': 'Critical', 'description': 'Service completely down, customer threatening to leave, regulatory compliance issue, or severe customer dissatisfaction'},
            {'label': 'High', 'description': 'Significant service degradation, billing dispute over large amount, repeated unresolved issue, or customer expressing strong frustration'},
            {'label': 'Medium', 'description': 'Standard service request, moderate billing question, upgrade inquiry, or general troubleshooting'},
            {'label': 'Low', 'description': 'Simple information request, minor account update, or routine inquiry with positive customer tone'}
        ],
        {
            'task_description': 'Classify call urgency based on issue severity and customer sentiment',
            'output_mode': 'single'
        }
    ) as urgency_classification,
    
    -- Classify by department
    AI_CLASSIFY(
        RAW_TRANSCRIPT:text::string,
        [
            {'label': 'Billing & Payments', 'description': 'Bill questions, payment issues, charges, refunds, pricing inquiries, or payment method updates'},
            {'label': 'Technical Support', 'description': 'Device issues, network problems, connectivity, settings, troubleshooting, or technical configuration'},
            {'label': 'Sales & Upgrades', 'description': 'New services, phone upgrades, plan changes, add-ons, or product information'},
            {'label': 'Retention & Loyalty', 'description': 'Cancellation requests, competitive offers, loyalty discounts, or contract negotiations'},
            {'label': 'Customer Complaints', 'description': 'Service complaints, poor experience, escalation requests, or formal complaint filing'},
            {'label': 'Account Management', 'description': 'Account updates, SIM replacement, address changes, or general account maintenance'}
        ],
        {
            'task_description': 'Route call to appropriate department based on primary issue',
            'output_mode': 'single'
        }
    ) as department_classification,
    
    -- Classify by complexity
    AI_CLASSIFY(
        RAW_TRANSCRIPT:text::string,
        [
            {'label': 'Simple - Self-Service', 'description': 'Could be handled by automated system or basic FAQ, single-step resolution'},
            {'label': 'Moderate - Standard Agent', 'description': 'Requires agent assistance but follows standard procedures, resolved in single call'},
            {'label': 'Complex - Senior Agent', 'description': 'Requires specialized knowledge, multiple systems, or policy exceptions'},
            {'label': 'Escalation Required', 'description': 'Needs supervisor approval, legal review, or senior management involvement'}
        ],
        {
            'task_description': 'Assess call complexity for appropriate resource allocation',
            'output_mode': 'single'
        }
    ) as complexity_classification
    
FROM {{transcripts_table}}
LIMIT 100;


In [ ]:
# Analyze call classifications
st.markdown("### 🎯 Call Classification Analysis")

class_df = AI_CLASSIFY.to_pandas()

# Parse classification results
def extract_label(json_str):
    try:
        data = json.loads(json_str)
        labels = data.get('labels', [])
        return labels[0] if labels else 'Unknown'
    except:
        return 'Unknown'

class_df['urgency'] = class_df['URGENCY_CLASSIFICATION'].apply(extract_label)
class_df['department'] = class_df['DEPARTMENT_CLASSIFICATION'].apply(extract_label)
class_df['complexity'] = class_df['COMPLEXITY_CLASSIFICATION'].apply(extract_label)

# Display distribution charts
col1, col2, col3 = st.columns(3)

with col1:
    st.markdown("#### Urgency Distribution")
    urgency_chart = alt.Chart(class_df).mark_arc().encode(
        theta=alt.Theta('count():Q'),
        color=alt.Color('urgency:N', 
                       scale=alt.Scale(domain=['Critical', 'High', 'Medium', 'Low'],
                                     range=['#d62728', '#ff7f0e', '#ffbb78', '#98df8a'])),
        tooltip=['urgency', 'count()']
    ).properties(height=200)
    st.altair_chart(urgency_chart, use_container_width=True)

with col2:
    st.markdown("#### Department Distribution")
    dept_chart = alt.Chart(class_df).mark_bar().encode(
        y=alt.Y('department:N', title='', sort='-x'),
        x=alt.X('count():Q', title='Calls'),
        color=alt.Color('department:N', legend=None),
        tooltip=['department', 'count()']
    ).properties(height=200)
    st.altair_chart(dept_chart, use_container_width=True)

with col3:
    st.markdown("#### Complexity Distribution")
    complexity_chart = alt.Chart(class_df).mark_arc().encode(
        theta=alt.Theta('count():Q'),
        color=alt.Color('complexity:N', scale=alt.Scale(scheme='blues')),
        tooltip=['complexity', 'count()']
    ).properties(height=200)
    st.altair_chart(complexity_chart, use_container_width=True)

# Priority matrix
st.markdown("### 📊 Priority Matrix: Urgency vs Complexity")
priority_matrix = class_df.groupby(['urgency', 'complexity']).size().reset_index(name='count')
heatmap = alt.Chart(priority_matrix).mark_rect().encode(
    x=alt.X('complexity:N', title='Complexity'),
    y=alt.Y('urgency:N', title='Urgency'),
    color=alt.Color('count:Q', scale=alt.Scale(scheme='blues'), title='Call Count'),
    tooltip=['urgency', 'complexity', 'count']
).properties(height=300)
st.altair_chart(heatmap, use_container_width=True)


# 😊 Customer Sentiment Analysis with AI_SENTIMENT

Understanding customer satisfaction in real-time is critical for retention and service quality. Using `AI_SENTIMENT`, we can analyze the emotional tone of conversations to identify at-risk customers, coaching opportunities, and service excellence.

This provides instant feedback on customer experience without waiting for post-call surveys.

## 🎯 Sentiment Insights:
- **Overall Call Sentiment**: Positive, Neutral, Negative scores
- **Sentiment Trajectory**: How sentiment changed during the call
- **Agent Performance**: Impact of agent responses on customer satisfaction
- **Risk Indicators**: Early warning signs of customer churn


In [ ]:
-- Comprehensive sentiment analysis of customer calls
-- This analyzes both overall sentiment and customer-specific sentiment

WITH call_segments AS (
    SELECT 
        AUDIO_FILE:RELATIVE_PATH::string as call_id,
        RAW_TRANSCRIPT:text::string as full_transcript,
        RAW_TRANSCRIPT:audio_duration::float as call_duration,
        -- Extract customer segments (SPEAKER_01 is typically the customer)
        ARRAY_TO_STRING(
            ARRAY_AGG(
                CASE 
                    WHEN seg.value:speaker_label = 'SPEAKER_01' 
                    THEN seg.value:text::string 
                END
            ) WITHIN GROUP (ORDER BY seg.index), 
            ' '
        ) as customer_text
    FROM {{transcripts_table}},
    LATERAL FLATTEN(input => RAW_TRANSCRIPT:segments) seg
    GROUP BY call_id, full_transcript, call_duration
)
SELECT 
    call_id,
    call_duration,
    
    -- Overall call sentiment
    TRIM(AI_SENTIMENT(full_transcript):categories[0].sentiment) as overall_sentiment,
    
    -- Customer-specific sentiment (more indicative of satisfaction)
    TRIM(AI_SENTIMENT(customer_text):categories[0].sentiment) as customer_sentiment,
    
    -- Analyze sentiment of opening (first 20% of call)
    TRIM(AI_SENTIMENT(SUBSTRING(full_transcript, 1, LENGTH(full_transcript) * 0.2)):categories[0].sentiment) as opening_sentiment,
    
    -- Analyze sentiment of closing (last 20% of call)
    TRIM(AI_SENTIMENT(SUBSTRING(full_transcript, LENGTH(full_transcript) * 0.8)):categories[0].sentiment) as closing_sentiment,
    
    full_transcript,
    customer_text
    
FROM call_segments
LIMIT 100;


In [ ]:
# Visualize sentiment analysis results
st.markdown("### 😊 Customer Sentiment Analysis")

sentiment_df = AI_SENTIMENT.to_pandas()

# Display key metrics
col1, col2, col3, col4 = st.columns(4)
with col1:
    avg_customer_sentiment = (sentiment_df['CUSTOMER_SENTIMENT'] == 'positive').mean()
    st.metric("Positive Sentiment Rate", f"{avg_customer_sentiment:.1%}")
with col2:
    positive_calls = (sentiment_df['CUSTOMER_SENTIMENT'] == 'positive').sum()
    st.metric("Positive Calls", f"{positive_calls} ({positive_calls/len(sentiment_df)*100:.0f}%)")
with col3:
    negative_calls = (sentiment_df['CUSTOMER_SENTIMENT'] == 'negative').sum()
    st.metric("Negative Calls", f"{negative_calls} ({negative_calls/len(sentiment_df)*100:.0f}%)")
with col4:
    improved_calls = ((sentiment_df['CLOSING_SENTIMENT'] == 'positive') & 
                     (sentiment_df['OPENING_SENTIMENT'] == 'negative')).sum()
    st.metric("Improved During Call", f"{improved_calls} ({improved_calls/len(sentiment_df)*100:.0f}%)")

# Sentiment distribution
st.markdown("### 📊 Sentiment Distribution")
sentiment_hist = alt.Chart(sentiment_df).mark_bar().encode(
    x=alt.X('CUSTOMER_SENTIMENT:N', title='Customer Sentiment'),
    y=alt.Y('count():Q', title='Number of Calls'),
    color=alt.Color('CUSTOMER_SENTIMENT:N', 
                   scale=alt.Scale(domain=['positive', 'neutral', 'negative'],
                                 range=['#2ca02c', '#7f7f7f', '#d62728'])),
    tooltip=['CUSTOMER_SENTIMENT', 'count()']
).properties(height=300)
st.altair_chart(sentiment_hist, use_container_width=True)

# Sentiment trajectory analysis
st.markdown("### 📈 Sentiment Trajectory: Opening vs Closing")
trajectory_data = sentiment_df[['CALL_ID', 'OPENING_SENTIMENT', 'CLOSING_SENTIMENT']].melt(
    id_vars=['CALL_ID'], 
    var_name='Stage', 
    value_name='Sentiment'
)
trajectory_data['Stage'] = trajectory_data['Stage'].map({
    'OPENING_SENTIMENT': 'Opening',
    'CLOSING_SENTIMENT': 'Closing'
})

trajectory_chart = alt.Chart(trajectory_data).mark_bar().encode(
    x=alt.X('Stage:N', title='Call Stage'),
    y=alt.Y('count():Q', title='Number of Calls'),
    color=alt.Color('Sentiment:N',
                   scale=alt.Scale(domain=['positive', 'neutral', 'negative'],
                                 range=['#2ca02c', '#7f7f7f', '#d62728'])),
    tooltip=['Stage', 'Sentiment', 'count()']
).properties(height=300)
st.altair_chart(trajectory_chart, use_container_width=True)

# At-risk customer identification
st.markdown("### ⚠️ At-Risk Customers (Negative Sentiment)")
at_risk = sentiment_df[sentiment_df['CUSTOMER_SENTIMENT'] == 'negative'].sort_values('CALL_ID')
if len(at_risk) > 0:
    st.dataframe(
        at_risk[['CALL_ID', 'CUSTOMER_SENTIMENT', 'OPENING_SENTIMENT', 'CLOSING_SENTIMENT', 'CALL_DURATION']]
        .head(10),
        use_container_width=True
    )
else:
    st.success("No at-risk customers identified in this sample! 🎉")


# 🎫 Automated Case Management System

The first step in production-ready call center intelligence is automating case creation from calls. Using a **Dynamic Table**, we'll create a continuously updated case management system that:

- Automatically processes new calls as they arrive
- Extracts key information using AI
- Assigns urgency levels and routing
- Tracks sentiment and resolution status
- Maintains an audit trail for compliance

Dynamic Tables in Snowflake automatically refresh based on changes to source data, making this a truly automated pipeline.


In [ ]:
-- Create automated case management system with Dynamic Table
-- This processes calls and creates cases automatically as new calls arrive
-- Target lag controls how fresh the data is (adjust based on your needs)

CREATE OR REPLACE DYNAMIC TABLE CALL_CENTER_CASES
  TARGET_LAG = '1 day'
  WAREHOUSE = tc_wh  -- Update to your warehouse
  AS
WITH call_processing AS (
    SELECT 
        AUDIO_FILE:RELATIVE_PATH::string as call_id,
        RAW_TRANSCRIPT:audio_duration::float as call_duration_seconds,
        RAW_TRANSCRIPT:text::string as transcript,
        
        -- Extract key entities using array format with detailed questions
        AI_EXTRACT(
            text => RAW_TRANSCRIPT:text::string,
            responseFormat => [
                ['agent_name', 'What is the name of the customer service agent handling this call?'],
                ['customer_issue_type', 'What was the primary reason for the call? (e.g., billing, technical support, account inquiry, complaint, upgrade)'],
                ['issue_description', 'Provide a brief but comprehensive description of the customer issue or request.'],
                ['resolution_provided', 'Was the customer issue fully resolved during this call? (True/False)'],
                ['resolution_details', 'Describe how the issue was resolved or what next steps were provided to the customer.'],
                ['products_mentioned', 'List all products or services discussed during the call (e.g., data plans, devices, roaming, insurance).'],
                ['follow_up_required', 'Is follow-up action required after this call? (True/False)'],
                ['follow_up_details', 'If follow-up is required, what specific actions need to be taken and by when?'],
                ['escalation_occurred', 'Was this call escalated to a supervisor or did the customer request escalation? (True/False)']
            ]
        ) as entities,
        
        -- Classify urgency
        AI_CLASSIFY(
            RAW_TRANSCRIPT:text::string,
            [
                {'label': 'Critical', 'description': 'Service down, customer threatening to leave, severe dissatisfaction'},
                {'label': 'High', 'description': 'Significant issue, billing dispute, repeated problem, strong frustration'},
                {'label': 'Medium', 'description': 'Standard service request, moderate question, general troubleshooting'},
                {'label': 'Low', 'description': 'Simple inquiry, minor update, routine question'}
            ],
            {'task_description': 'Classify urgency level', 'output_mode': 'single'}
        ) as urgency_class,
        
        -- Classify department
        AI_CLASSIFY(
            RAW_TRANSCRIPT:text::string,
            [
                {'label': 'Billing', 'description': 'Bill questions, payment issues, charges, refunds'},
                {'label': 'Technical Support', 'description': 'Device issues, network problems, connectivity, troubleshooting'},
                {'label': 'Sales', 'description': 'New services, upgrades, plan changes, product info'},
                {'label': 'Retention', 'description': 'Cancellation requests, competitive offers, loyalty'},
                {'label': 'Complaints', 'description': 'Service complaints, poor experience, escalation requests'}
            ],
            {'task_description': 'Route to department', 'output_mode': 'single'}
        ) as department_class,
        
        -- Analyze sentiment
        TRIM(AI_SENTIMENT(RAW_TRANSCRIPT:text::string):categories[0].sentiment) as customer_sentiment
        
    FROM {{transcripts_table}}
)
SELECT 
    call_id as CALL_ID,
    entities:response.agent_name::string as AGENT_NAME,
    entities:response.customer_issue_type::string as ISSUE_TYPE,
    entities:response.issue_description::string as ISSUE_DESCRIPTION,
    entities:response.resolution_provided::boolean as RESOLVED,
    entities:response.resolution_details::string as RESOLUTION_DETAILS,
    entities:response.products_mentioned as PRODUCTS_MENTIONED,
    entities:response.follow_up_required::boolean as FOLLOW_UP_REQUIRED,
    entities:response.follow_up_details::string as FOLLOW_UP_DETAILS,
    entities:response.escalation_occurred::boolean as ESCALATED,
    urgency_class:labels[0]::string as URGENCY_LEVEL,
    department_class:labels[0]::string as DEPARTMENT,
    customer_sentiment as SENTIMENT_SCORE,
    CASE 
        WHEN lower(customer_sentiment) = 'negative' THEN 'At Risk'
        WHEN lower(customer_sentiment) = 'positive' THEN 'Satisfied'
        ELSE 'Neutral'
    END as CUSTOMER_STATUS,
    ROUND(call_duration_seconds / 60, 2) as DURATION_MINUTES,
    CASE 
        WHEN entities:response.resolution_provided::boolean = TRUE THEN 'Closed'
        WHEN entities:response.follow_up_required::boolean = TRUE THEN 'Pending Follow-up'
        WHEN entities:response.escalation_occurred::boolean = TRUE THEN 'Escalated'
        ELSE 'Open'
    END as CASE_STATUS,
    transcript
FROM call_processing;


In [ ]:
SELECT * FROM CALL_CENTER_CASES LIMIT 10;

# 🔍 Cortex Search for Historical Call Intelligence

Now let's enable **semantic search** across all call transcripts. This allows agents to instantly find similar past issues, proven resolutions, and best practices from thousands of historical calls.

Unlike keyword search, Cortex Search understands meaning and context—finding relevant calls even when they use different words to describe the same problem.

## 🎯 Use Cases:
- **"Show me calls about roaming charges in Europe"** → Finds all related calls, even if they mention "international data" or "travel fees"
- **"How did agents handle angry billing complaints?"** → Returns calls with similar situations and their resolutions
- **"Find technical issues with iPhone 15"** → Surfaces device-specific problems and solutions


In [ ]:
-- Create Cortex Search service for semantic search across call transcripts
-- This indexes call data for fast, intelligent retrieval based on meaning

CREATE OR REPLACE CORTEX SEARCH SERVICE CALL_CENTER_SEARCH
  ON TRANSCRIPT
  WAREHOUSE = tc_wh  -- Update to your warehouse
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
  AS (
    SELECT
        AUDIO_FILE:RELATIVE_PATH::string as call_id,
        RAW_TRANSCRIPT:text::string as transcript
    FROM {{transcripts_table}}
);


In [ ]:
# Test Cortex Search with sample queries
st.header("🔍 Semantic Call Search")

st.markdown("""
Search across all historical calls using natural language. The system understands meaning, not just keywords.
""")

# Sample search queries to try
st.markdown("**💡 Try these example searches:**")
col1, col2 = st.columns(2)
with col1:
    st.code("roaming charges complaints")
    st.code("billing disputes with angry customers")
    st.code("network connectivity problems")
with col2:
    st.code("successful retention of canceling customer")
    st.code("iPhone technical support issues")
    st.code("payment plan negotiations")

with st.form("search_form"):
    search_query = st.text_input(
        "🔎 Search historical calls:",
        placeholder="e.g., 'How were billing disputes resolved successfully?'"
    )
    
    col1, col2 = st.columns(2)
    with col1:
        limit_results = st.slider("Number of results:", 3, 20, 5)
    with col2:
        filter_dept = st.selectbox(
            "Filter by department (optional):",
            ["All Departments", "Billing", "Technical Support", "Sales", "Retention", "Complaints"]
        )
    
    search_submitted = st.form_submit_button("Search")

if search_submitted and search_query:
    try:
        from snowflake.core import Root
        
        root = Root(session)
        search_service = (root
            .databases["ai_sol"]
            .schemas["call_centers"]
            .cortex_search_services["call_center_search"]
        )
        
        # Build filter if department selected
        filter_obj = None
        if filter_dept != "All Departments":
            filter_obj = {"@eq": {"DEPARTMENT": filter_dept}}
        
        # Perform search
        if filter_obj:
            search_results = search_service.search(
                query=search_query,
                columns=["CALL_ID", "TRANSCRIPT"],
                limit=limit_results,
                filter=filter_obj
            )
        else:
            search_results = search_service.search(
                query=search_query,
                columns=["CALL_ID", "TRANSCRIPT"],
                limit=limit_results
            )
        
        # Parse results
        results_json = json.loads(search_results.to_json())
        
        if results_json and 'results' in results_json and len(results_json['results']) > 0:
            st.success(f"Found {len(results_json['results'])} relevant calls")
            
            # Display each result
            for i, result in enumerate(results_json['results']):
                call_id = result.get('CALL_ID', 'N/A')
                transcript = result.get('TRANSCRIPT', 'No transcript available')
                
                with st.expander(f"🎫 Call {i+1}: {call_id}"):
                    st.markdown(f"**Call ID:** {call_id}")
                    
                    st.markdown("**Transcript:**")
                    st.text_area(
                        label="Full Transcript", 
                        value=transcript, 
                        height=200, 
                        key=f"transcript_{i}",
                        label_visibility="collapsed"
                    )
                    
                    # Generate AI summary for this transcript
                    with st.spinner("Generating AI summary..."):
                        summary_prompt = f"""Provide a concise summary of this call center transcript. Include:
1. Main issue or reason for the call
2. Key points discussed
3. Outcome or resolution

Transcript:
{transcript[:3000]}"""  # Limit transcript length for API
                        
                        try:
                            summary_result = session.sql(f"""
                                SELECT AI_COMPLETE('claude-sonnet-4-5', $${summary_prompt}$$) as summary
                            """).collect()
                            
                            if summary_result:
                                st.markdown("**🤖 AI Summary:**")
                                st.info(summary_result[0]['SUMMARY'])
                            else:
                                st.warning("Could not generate summary")
                        except Exception as e:
                            st.warning(f"Summary generation unavailable: {str(e)}")
        else:
            st.warning("No results found. Try a different search query or check that the search service is ready.")
            
    except Exception as e:
        st.error(f"Search service not ready or error occurred: {str(e)}")
        st.info("Make sure the CALL_CENTER_SEARCH service has been created and has finished indexing.")


# 📊 Semantic View for Natural Language Analytics

Semantic Views enable **Cortex Analyst** to understand your call center data and answer business questions in natural language. This empowers managers and executives to explore data conversationally without writing SQL.

By defining relationships, metrics, and business context, we create a semantic layer that translates questions like *"What's the average resolution time for billing issues this month?"* into accurate SQL queries.

## 🎯 Key Benefits:
- **Self-Service Analytics**: Non-technical users can query data naturally
- **Consistent Metrics**: Everyone uses the same definitions
- **Data Governance**: Controlled access through semantic layer
- **Business Context**: AI understands your domain-specific terminology


In [ ]:
-- Create semantic view for call center analytics
-- This enables Cortex Analyst to understand and query your data naturally

CREATE OR REPLACE SEMANTIC VIEW call_center_analytics
TABLES (
  cases AS AI_SOL.CALL_CENTERS.CALL_CENTER_CASES 
    PRIMARY KEY (CALL_ID) 
    WITH SYNONYMS ('calls', 'tickets', 'interactions', 'customer contacts')
    COMMENT = 'Call center cases with AI-extracted intelligence'
)
FACTS (
  cases.call_count AS 1 COMMENT = 'Count of individual calls/cases',
  cases.duration_minutes AS DURATION_MINUTES COMMENT = 'Call duration in minutes'
)
DIMENSIONS (
  cases.CALL_ID AS CALL_ID 
    WITH SYNONYMS ('ticket id', 'call id', 'case number') 
    COMMENT = 'Unique identifier for each case',
  
  cases.agent_name AS AGENT_NAME 
    WITH SYNONYMS ('agent', 'representative', 'support agent', 'rep')
    COMMENT = 'Name of the agent who handled the call',
  
  cases.issue_type AS ISSUE_TYPE 
    WITH SYNONYMS ('problem type', 'issue category', 'call reason')
    COMMENT = 'Category of customer issue or inquiry',
  
  cases.issue_description AS ISSUE_DESCRIPTION 
    WITH SYNONYMS ('problem description', 'issue details')
    COMMENT = 'Detailed description of the customer issue',
  
  cases.department AS DEPARTMENT 
    WITH SYNONYMS ('team', 'group', 'department name')
    COMMENT = 'Department that handled the call',
  
  cases.urgency_level AS URGENCY_LEVEL 
    WITH SYNONYMS ('priority', 'urgency', 'severity')
    COMMENT = 'Urgency classification: Critical, High, Medium, Low',
  
  cases.case_status AS CASE_STATUS 
    WITH SYNONYMS ('status', 'state', 'case state')
    COMMENT = 'Current status of the case',
  
  cases.customer_status AS CUSTOMER_STATUS 
    WITH SYNONYMS ('satisfaction', 'customer state', 'satisfaction level')
    COMMENT = 'Customer satisfaction status: At Risk, Neutral, Satisfied',
  
  cases.resolved AS RESOLVED 
    WITH SYNONYMS ('is resolved', 'resolved flag', 'closed')
    COMMENT = 'Whether the issue was resolved',
  
  cases.follow_up_required AS FOLLOW_UP_REQUIRED 
    WITH SYNONYMS ('needs follow up', 'requires follow up')
    COMMENT = 'Whether follow-up action is needed',
  
  cases.escalated AS ESCALATED 
    WITH SYNONYMS ('is escalated', 'escalation flag')
    COMMENT = 'Whether the call was escalated'
)
METRICS (
  cases.total_calls AS COUNT(cases.CALL_ID) 
    COMMENT = 'Total number of calls/cases',
  
  cases.resolved_calls AS COUNT(
    CASE WHEN cases.RESOLVED = TRUE THEN cases.CALL_ID END
  ) 
    COMMENT = 'Number of resolved calls',
  
  cases.unresolved_calls AS COUNT(
    CASE WHEN cases.RESOLVED = FALSE OR cases.RESOLVED IS NULL THEN cases.CALL_ID END
  ) 
    COMMENT = 'Number of unresolved calls',
  
  cases.resolution_rate AS (
    COUNT(CASE WHEN cases.RESOLVED = TRUE THEN cases.CALL_ID END) * 100.0 / 
    COUNT(cases.CALL_ID)
  ) 
    COMMENT = 'Percentage of calls that were resolved',
  
  cases.critical_calls AS COUNT(
    CASE WHEN cases.URGENCY_LEVEL = 'Critical' THEN cases.CALL_ID END
  ) 
    COMMENT = 'Number of critical priority calls',
  
  cases.high_priority_calls AS COUNT(
    CASE WHEN cases.URGENCY_LEVEL IN ('Critical', 'High') THEN cases.CALL_ID END
  ) 
    COMMENT = 'Number of high priority calls',
  
  cases.at_risk_customers AS COUNT(
    CASE WHEN cases.CUSTOMER_STATUS = 'At Risk' THEN cases.CALL_ID END
  ) 
    COMMENT = 'Number of at-risk customer interactions',
  
  cases.satisfied_customers AS COUNT(
    CASE WHEN cases.CUSTOMER_STATUS = 'Satisfied' THEN cases.CALL_ID END
  ) 
    COMMENT = 'Number of satisfied customer interactions',
  
  cases.escalated_calls AS COUNT(
    CASE WHEN cases.ESCALATED = TRUE THEN cases.CALL_ID END
  ) 
    COMMENT = 'Number of escalated calls',
  
  cases.escalation_rate AS (
    COUNT(CASE WHEN cases.ESCALATED = TRUE THEN cases.CALL_ID END) * 100.0 / 
    COUNT(cases.CALL_ID)
  ) 
    COMMENT = 'Percentage of calls that were escalated',
  
  cases.avg_call_duration AS AVG(cases.DURATION_MINUTES) 
    COMMENT = 'Average call duration in minutes',
  
  cases.total_call_minutes AS SUM(cases.DURATION_MINUTES) 
    COMMENT = 'Total call time in minutes',
  
  cases.avg_sentiment AS AVG(cases.SENTIMENT_SCORE) 
    COMMENT = 'Average customer sentiment score (-1 to 1)',
  
  cases.calls_needing_followup AS COUNT(
    CASE WHEN cases.FOLLOW_UP_REQUIRED = TRUE THEN cases.CALL_ID END
  ) 
    COMMENT = 'Number of calls requiring follow-up'
)
COMMENT = 'Semantic view for conversational call center analytics with Cortex Analyst';


In [ ]:
-- Test the semantic view with a simple query
-- Semantic views can be queried directly or used by Cortex Analyst
-- This example shows resolution rates by department

SELECT * FROM SEMANTIC_VIEW(
    call_center_analytics
    DIMENSIONS cases.department, cases.urgency_level
    METRICS total_calls, escalated_calls, avg_call_duration
)
ORDER BY total_calls DESC;


# 🚀 Next Steps: From Demo to Production

Congratulations! You've experienced the power of AI-driven call center intelligence. Here's how to take this from demo to production:

## 1. 🔒 **Deploy Privacy Pipeline**
Implement the PII removal process as a standard preprocessing step for all call transcripts, ensuring compliance with GDPR, CCPA, and other regulations.

## 2. 📊 **Expand Semantic Views**
Head to **AI > Studio > Cortex Analyst** to create semantic views of your call center data, enabling natural language reporting for managers and executives.

## 3. 🤖 **Build Agent Assist Tools**
Deploy the Streamlit agent assist application to provide real-time intelligence, response suggestions, and quality coaching during live calls.

## 4. 🔄 **Automate Case Management**
Connect AI extraction and classification to your CRM or ticketing system for automatic case creation, routing, and follow-up tracking.

## 5. 📈 **Scale Across Contact Centers**
Roll out to your entire organization with:
- Team-specific performance dashboards
- Real-time quality assurance monitoring
- Predictive analytics for staffing and training
- Customer experience optimization

---

### 💡 **Key Takeaways**

✅ **Privacy-First AI** ensures compliance while extracting maximum value from call data

✅ **Automated Intelligence** transforms hours of manual analysis into instant insights

✅ **Smart Triaging** routes calls to the right resources at the right time

✅ **Sentiment Analysis** provides real-time customer satisfaction monitoring

✅ **Agent Augmentation** empowers teams with AI-powered assistance and coaching

---

*Ready to revolutionize your call center operations? Start with your own call transcript data and see the transformation firsthand.*
